# Construcción del modelo en mediapipe

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Hermanos\Desktop\Proyecto Deepfake\.venv-mediapipe\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
print("¿CUDA disponible?", torch.cuda.is_available())
print("GPU actual:", torch.cuda.get_device_name(0))

¿CUDA disponible? True
GPU actual: NVIDIA GeForce RTX 4070 Ti


In [22]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.utils import shuffle
from collections import Counter

class DeepfakeDataset(Dataset):
    def __init__(self, base_path, sequence_length=16, split='train', val_split=0.2):
        self.sequence_length = sequence_length
        self.base_path = base_path
        self.transform = transforms.Compose([
            transforms.ToTensor(),
        ])
        
        # Paso 1: Cargar videos completos con etiquetas
        all_videos = self._collect_video_ids(base_path)
        all_videos = shuffle(all_videos, random_state=42)  # Mezcla antes de dividir
        
        # Paso 2: Dividir por video, no por muestra
        split_idx = int(len(all_videos) * (1 - val_split))
        self.videos = all_videos[:split_idx] if split == 'train' else all_videos[split_idx:]

    def _collect_video_ids(self, base_path):
        video_samples = []
        for folder in os.listdir(base_path):
            folder_path = os.path.join(base_path, folder)
            label = 0 if folder == "original" else 1
            video_ids = {}
            for file in os.listdir(folder_path):
                if file.endswith(".jpg"):
                    vid = "_".join(file.split("_")[:-1])
                    video_ids.setdefault(vid, []).append(file)
            for vid, files in video_ids.items():
                if len(files) >= self.sequence_length:
                    video_samples.append((folder_path, vid, label))
        return video_samples

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        folder, video_id, label = self.videos[idx]
        imgs = []
        lmks = []
        for i in range(self.sequence_length):
            img_path = os.path.join(folder, f"{video_id}_{i}.jpg")
            lmk_path = os.path.join(folder, f"{video_id}_{i}.npy")
            img = cv2.imread(img_path)
            img = cv2.resize(img, (256, 256))
            img = self.transform(img)  # (3, 256, 256)
            lmk = torch.tensor(np.load(lmk_path), dtype=torch.float32)
            imgs.append(img)
            lmks.append(lmk)
        imgs = torch.stack(imgs)      # (16, 3, 256, 256)
        lmks = torch.stack(lmks)      # (16, 5)
        return imgs, lmks, torch.tensor(label, dtype=torch.float32)
    

# ✅ Diagnóstico: Conteo de clases reales/falsas
def print_class_distribution(dataset, name):
    labels = [int(label.item()) for _, _, label in dataset]
    counts = Counter(labels)
    print(f"{name} class distribution:")
    print(f"  Real (0): {counts.get(0, 0)}")
    print(f"  Fake (1): {counts.get(1, 0)}")
    print("-" * 30)


In [23]:
import torchvision.models as models
import torch.nn as nn

class DeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        self.cnn.classifier = nn.Identity()  # elimina la capa final
        self.embedding_dim = 1280
        self.sequence_length = 16

        self.lstm = nn.LSTM(input_size=1285, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x_imgs, x_lmks):
        B, T, C, H, W = x_imgs.shape
        x_imgs = x_imgs.view(B * T, C, H, W)
        features = self.cnn(x_imgs)                     # (B*T, 1280)
        features = features.view(B, T, -1)              # (B, T, 1280)
        combined = torch.cat([features, x_lmks], dim=2) # (B, T, 1285)
        out, _ = self.lstm(combined)
        out = out[:, -1, :]                             # última salida
        return self.fc(out).squeeze(1)


In [24]:
base_path = r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesamiento_mediapipe"

train_dataset = DeepfakeDataset(base_path, split='train')
val_dataset = DeepfakeDataset(base_path, split='val')

print_class_distribution(train_dataset, "Train")
print_class_distribution(val_dataset, "Validation")


Train class distribution:
  Real (0): 768
  Fake (1): 723
------------------------------
Validation class distribution:
  Real (0): 207
  Fake (1): 166
------------------------------


In [25]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from sklearn.metrics import accuracy_score, f1_score
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset y loaders
train_dataset = DeepfakeDataset(base_path=r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesamiento_mediapipe", split='train')
val_dataset = DeepfakeDataset(base_path=r"C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesamiento_mediapipe", split='val')

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Modelo
model = DeepfakeDetector().to(device)
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.BCELoss()

best_f1 = 0.0

for epoch in range(1, 11):
    model.train()
    running_loss = 0.0
    for imgs, lmks, labels in train_loader:
        imgs, lmks, labels = imgs.to(device), lmks.to(device), labels.to(device)
        preds = model(imgs, lmks)
        loss = criterion(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # Validación
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, lmks, labels in val_loader:
            imgs, lmks = imgs.to(device), lmks.to(device)
            outputs = model(imgs, lmks)
            preds = (outputs > 0.5).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Epoch {epoch}: Loss={avg_loss:.4f} | Val Accuracy={acc:.4f} | Val F1-score={f1:.4f}")

    # Guardar mejor modelo
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "mediapipe_model.pth")
        print(f"✅ Modelo guardado con F1-score = {f1:.4f}")


Epoch 1: Loss=0.6595 | Val Accuracy=0.7507 | Val F1-score=0.7424
✅ Modelo guardado con F1-score = 0.7424
Epoch 2: Loss=0.5283 | Val Accuracy=0.8874 | Val F1-score=0.8750
✅ Modelo guardado con F1-score = 0.8750
Epoch 3: Loss=0.3916 | Val Accuracy=0.8740 | Val F1-score=0.8698
Epoch 4: Loss=0.3354 | Val Accuracy=0.9008 | Val F1-score=0.8969
✅ Modelo guardado con F1-score = 0.8969
Epoch 5: Loss=0.2427 | Val Accuracy=0.9303 | Val F1-score=0.9217
✅ Modelo guardado con F1-score = 0.9217
Epoch 6: Loss=0.1966 | Val Accuracy=0.7909 | Val F1-score=0.8079
Epoch 7: Loss=0.1488 | Val Accuracy=0.9491 | Val F1-score=0.9408
✅ Modelo guardado con F1-score = 0.9408
Epoch 8: Loss=0.1300 | Val Accuracy=0.9357 | Val F1-score=0.9245
Epoch 9: Loss=0.0682 | Val Accuracy=0.9196 | Val F1-score=0.9143
Epoch 10: Loss=0.0860 | Val Accuracy=0.9357 | Val F1-score=0.9264


In [26]:

from collections import Counter
print("Preds:", Counter(np.array(y_pred, dtype=int)))

Preds: Counter({0: 213, 1: 160})


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=["real", "fake"]))

              precision    recall  f1-score   support

        real       0.93      0.96      0.94       207
        fake       0.94      0.91      0.93       166

    accuracy                           0.94       373
   macro avg       0.94      0.93      0.93       373
weighted avg       0.94      0.94      0.94       373

